In [2]:
import pandas as pd
import torch
import numpy as np 

In [12]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [13]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [14]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [15]:
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.3739,  0.7924,  0.7507],
        [-1.2867, -0.3299, -0.5902]], requires_grad=True)
tensor([0.7640, 1.3178], requires_grad=True)


In [18]:
#model multiplies the inputs wit hthe weights and adds a bias
def model(x):
    return x @ w.t() + b

In [25]:
preds = model(in_torch)
preds

tensor([[  58.8419, -140.0949],
        [  84.5175, -182.5780],
        [ 117.9615, -189.0653],
        [  24.4751, -165.9510],
        [ 103.5877, -160.4505]], grad_fn=<AddBackward0>)

In [19]:
# MSE loss function
def mse(t1,t2):
    diff = t1 - t2
    # * multiplies each element with itself
    # numel gives the number of elements
    # gives mean squared error
    return torch.sum(diff * diff) / diff.numel()

In [26]:
loss = mse(preds, targets)
loss

tensor(34759.2109, grad_fn=<DivBackward0>)

In [27]:
loss.backward()
# gradients are computed and stored in the .grad property for each variable

In [23]:
print(w)
print(w.grad)

tensor([[-0.3739,  0.7924,  0.7507],
        [-1.2867, -0.3299, -0.5902]], requires_grad=True)
tensor([[   146.0410,    104.7258,     83.9604],
        [-21829.0586, -23548.4219, -14586.7168]])


In [24]:
# need to call zero_() method to clear the gradients
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [28]:
# adjust weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [29]:
print(w)

tensor([[-0.3754,  0.7914,  0.7499],
        [-1.0684, -0.0944, -0.4443]], requires_grad=True)


In [30]:
# train model for 100 epochs

for i in range(1000):
    preds = model(in_torch)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [32]:
print(model(in_torch))
print(mse(preds, targets))

tensor([[ 57.2697,  70.7702],
        [ 82.3283,  98.5473],
        [118.2513, 137.0493],
        [ 20.9526,  38.1539],
        [102.3572, 115.0432]], grad_fn=<AddBackward0>)
tensor(4.5515, grad_fn=<DivBackward0>)


In [33]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [35]:
# generalized example for larger datasets
import torch.nn as nn

In [36]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [38]:
# tensor dataset allows access to rwos from inputs and targets as tuples, used for working with large datasets that cannot be loaded into memory
from torch.utils.data import TensorDataset

In [39]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [40]:
# the dataloadder can split the data into batches, and has abilities to shuffle and random sample from the data
from torch.utils.data import DataLoader

In [41]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [42]:
# instead of initializing weights and biases manually, we can define the model using nn.Linear class from Pytorch
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0289, -0.2768, -0.0617],
        [ 0.4219,  0.0874,  0.2053]], requires_grad=True)
Parameter containing:
tensor([-0.4106, -0.0151], requires_grad=True)


In [44]:
# parameters method returns a list of all weights and biases
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0289, -0.2768, -0.0617],
         [ 0.4219,  0.0874,  0.2053]], requires_grad=True),
 Parameter containing:
 tensor([-0.4106, -0.0151], requires_grad=True)]

In [45]:
preds = model(inputs)
preds

tensor([[-19.5034,  45.4713],
        [-26.0929,  59.2133],
        [-38.5712,  60.3150],
        [-11.6522,  54.3775],
        [-29.3130,  51.8619],
        [-19.1977,  45.8058],
        [-25.8779,  59.3312],
        [-38.6041,  60.9422],
        [-11.9579,  54.0430],
        [-29.4037,  51.6453],
        [-19.2883,  45.5892],
        [-25.7873,  59.5478],
        [-38.7863,  60.1971],
        [-11.5616,  54.5942],
        [-29.6187,  51.5274]], grad_fn=<AddmmBackward>)

In [46]:
# we can use a built in loss function
import torch.nn.functional as F

In [47]:
loss_fn = F.mse_loss

In [48]:
loss_fn(model(inputs),targets)

tensor(7283.4863, grad_fn=<MseLossBackward>)

In [49]:
# instead of manually manipulating the model's weights and biases using gradients, we can use the SGD built in optimizer

In [51]:
# define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [61]:
# fit funtion trains the model given the parameters
def fit(num_epochs, model, loss_fn, opt):

    #repeat for a number of epochs
    for epoch in range(num_epochs):

        # train with batches of data
        for xb, yb in train_dl:

            # 1 generate prediction
            pred = model(xb)

            # 2 calculate loss
            loss = loss_fn(pred, yb)

            # 3 compute gradient
            loss.backward()

            # 4 update parameters using gradients
            opt.step()

            # 5 reset gradients to zero
            opt.zero_grad()
        
        if(epoch+1)%100 ==0:
            print("Epoch: {0}/{1}, loss: {2}".format(epoch,num_epochs,loss.item()))

In [62]:
fit(1000, model, loss_fn, opt)

Epoch: 99/1000, loss: 26.84378433227539
Epoch: 199/1000, loss: 6.273566246032715
Epoch: 299/1000, loss: 6.437229156494141
Epoch: 399/1000, loss: 2.884883165359497
Epoch: 499/1000, loss: 2.881657123565674
Epoch: 599/1000, loss: 1.1836341619491577
Epoch: 699/1000, loss: 1.9658706188201904
Epoch: 799/1000, loss: 0.9058719873428345
Epoch: 899/1000, loss: 0.9968701601028442
Epoch: 999/1000, loss: 1.1773329973220825


In [63]:
model(inputs)

tensor([[ 56.9007,  70.3974],
        [ 81.9516, 100.3443],
        [118.4793, 133.1394],
        [ 20.9014,  37.8391],
        [101.5890, 118.1169],
        [ 55.6546,  69.3155],
        [ 81.7874, 100.4080],
        [118.7667, 133.7240],
        [ 22.1474,  38.9210],
        [102.6708, 119.2625],
        [ 56.7364,  70.4611],
        [ 80.7056,  99.2624],
        [118.6436, 133.0757],
        [ 19.8196,  36.6936],
        [102.8350, 119.1988]], grad_fn=<AddmmBackward>)

In [64]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])